# Recurrent Neural Network Demo

In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, LeakyReLU, Input, GRU
from tensorflow.keras import backend

import random

import re

##################
# Default tick label size
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24

## Generate Data

In [ ]:

def generate_sequence(X, min_delay=1, max_delay=1):
    '''
    Generate a single sequence given a vectorial (column) cue
    
    The general sequence is:
    - CUE: 1 step
    - Wait with CUE: random number of steps
    - Movement with no CUE: 3 steps
    - Hold with no CUE: random number of steps
    '''
    # Initial state
    cue = X
    go = np.array([[0.0]])
    
    ins = np.vstack((cue, go))
    outs = np.zeros((X.shape[0], 1))
    
    # Wait period
    n = random.randint(min_delay, max_delay)+1
    
    # Set cue to zero (force network to remember)
    cue = np.zeros(cue.shape)
    
    # Loop over wait  period
    for i in range(n):
        ins_new = np.vstack((cue, go))
        outs_new = np.zeros((X.shape[0], 1))
        ins = np.hstack((ins, ins_new))
        outs = np.hstack((outs, outs_new))

    # Go: move step 1
    cue = np.zeros((cue.shape[0], 1))
    go = np.array([[1.0]])
    ins_new = np.vstack((cue, go))
    outs_new = X * 0.33
    ins = np.hstack((ins, ins_new))
    outs = np.hstack((outs, outs_new))
    
    # Move step 2
    go = np.array([[1.0]])
    ins_new = np.vstack((cue, go))
    outs_new = X * 0.67
    ins = np.hstack((ins, ins_new))
    outs = np.hstack((outs, outs_new))
    
    # Move step 3
    go = np.array([[1.0]])
    ins_new = np.vstack((cue, go))
    outs_new = X 
    ins = np.hstack((ins, ins_new))
    outs = np.hstack((outs, outs_new))
    
    # Repeat final step
    for i in range(10-n):
        ins = np.hstack((ins, ins_new))
        outs = np.hstack((outs, outs_new))
        
    return ins, outs

def generate_sequence_set_from_cues(cues, min_delay=1, max_delay=1):
    '''
    Generate a set of sequences
    
    Output format: SEQUENCES x STEPS x FEATURES
    '''
    
    # First one
    cue = np.array([[cues[0][0]],[cues[0][1]]])
    ins_single, outs_single = generate_sequence(cue, min_delay=min_delay, max_delay=max_delay)
    ins_single = ins_single.T
    outs_single = outs_single.T
    
    # Shape of a single trial
    ins_shape = ins_single.shape
    outs_shape = outs_single.shape
    
    # Change to 1 x STEPS x features
    ins = ins_single.reshape((1,ins_shape[0], ins_shape[1]))
    outs = outs_single.reshape((1,outs_shape[0], outs_shape[1]))
    
    ##################
    # Add the others
    for cue in cues[1:]:
        cue = np.array([[cue[0]],[cue[1]]])
        ins_single, outs_single = generate_sequence(cue, min_delay=min_delay, max_delay=max_delay)
        ins = np.insert(ins, ins.shape[0], ins_single.T, axis=0)
        outs = np.insert(outs, outs.shape[0], outs_single.T, axis=0)
    
    return ins, outs

def generate_full_sequence_set(min_delay=1, max_delay=6):
    '''
    Generate a set of sequences from a fixed set of goals
    
    Output format: SEQUENCES x STEPS x FEATURES
    '''
    
    cues = [[1, 0], [-1, 0], [0, 1], [0, -1],
            [1, 1], [-1, 1], [-1, 1], [-1, -1],
            [1, 0], [-1, 0], [0, 1], [0, -1],
            [1, 1], [-1, 1], [-1, 1], [-1, -1],
            [1, 0], [-1, 0], [0, 1], [0, -1],
            [1, 1], [-1, 1], [-1, 1], [-1, -1],
            [1, 0], [-1, 0], [0, 1], [0, -1],
            [1, 1], [-1, 1], [-1, 1], [-1, -1]]
    return generate_sequence_set_from_cues(cues, min_delay, max_delay)
    

In [ ]:
# training and validation sets
ins, outs = generate_full_sequence_set()
ins_validation, outs_validation = generate_full_sequence_set()

In [ ]:
ins[0,:,:]

In [ ]:
def symmetric_leaky_relu(X):
    '''
    This non-linearity "leaks" on both the positive and negative sides
    
    :param X: net input tensor
    :return: tensor that contains the output of the non-linearity
    '''
    return 1.0-LeakyReLU(0.1)(1.0-LeakyReLU(0.1)(X))
  

In [ ]:
def create_network(ins, outs, n_neurons=10, activation='tanh', 
                   activation_dense=None, lambda_regularization=0):
    ins_shape = ins.shape
    outs_shape = outs.shape
    
    model = Sequential()
    
    # Fill in
    
    
    model.add(Dense(units=int(n_neurons/2), activation=activation_dense,
                         kernel_initializer='random_uniform',
                           bias_initializer='random_uniform',
                            kernel_regularizer=keras.regularizers.l2(lambda_regularization),
                            bias_regularizer=keras.regularizers.l2(lambda_regularization)))
    
    model.add(Dense(units=2,
                   kernel_initializer='random_uniform',
                           bias_initializer='random_uniform',
                            kernel_regularizer=keras.regularizers.l2(lambda_regularization),
                            bias_regularizer=keras.regularizers.l2(lambda_regularization)))
    
              
    # The optimizer determines how the gradient descent is to be done
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, 
                            epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(loss='mse', optimizer=opt)
              
    return model

def create_network_gru(ins, outs, n_neurons=10, activation='tanh', activation_dense=None, lambda_regularization=0):
    ins_shape = ins.shape
    outs_shape = outs.shape
    
    model = Sequential()
    
    # Fill in
    
    model.add(Dense(units=int(n_neurons/2), activation=activation_dense,
                         kernel_initializer='random_uniform',
                           bias_initializer='random_uniform',
                            kernel_regularizer=keras.regularizers.l2(lambda_regularization),
                            bias_regularizer=keras.regularizers.l2(lambda_regularization)))
    
    model.add(Dense(units=2,
                   kernel_initializer='random_uniform',
                           bias_initializer='random_uniform',
                            kernel_regularizer=keras.regularizers.l2(lambda_regularization),
                            bias_regularizer=keras.regularizers.l2(lambda_regularization)))
    
              
    # The optimizer determines how the gradient descent is to be done
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, 
                            epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(loss='mse', optimizer=opt)
              
    return model


    

In [ ]:
def report_network(model, ins, outs, 
                  performance_log=[], sample=0):
    pred = model.predict(ins)
    
    fs =18
    # Plot sample
    fig = plt.figure(figsize=(12,6))
    
    # X
    plt.subplot(121)
    plt.plot(pred[sample,:,0], 'b.')
    plt.plot(outs[sample,:,0], 'r.')
    plt.ylim(-1.2,1.2)
    plt.title("X", fontsize=fs)
    plt.ylabel("position", fontsize=fs)
    
    # Y
    plt.subplot(122)
    plt.plot(pred[sample,:,1], 'b.')
    plt.plot(outs[sample,:,1], 'r.')
    plt.ylim(-1.2,1.2)
    plt.title("Y", fontsize=fs)



## Experiments

In [ ]:
model = create_network(ins, outs, n_neurons=40, 
                       activation=symmetric_leaky_relu, 
                       activation_dense=symmetric_leaky_relu, 
                       lambda_regularization=0.0001)

In [ ]:
history = model.fit(ins, outs, epochs=1000, verbose=0)

In [ ]:
report_network(model, ins_validation, outs_validation)
print(model.summary())

In [ ]:
model_gru = create_network_gru(ins, outs, n_neurons=40, 
                       activation_dense=symmetric_leaky_relu, lambda_regularization=0.0001)

In [ ]:
history = model_gru.fit(ins, outs, epochs=1000, verbose=0)

In [ ]:
report_network(model_gru, ins_validation, outs_validation)
print(model_gru.summary())

In [ ]:
# In case we want to clear out all of the older models
backend.clear_session()